# Guild AI Interactive Python Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://my.guild.ai/start).

Guild AI is a tool for running, tracking, and comparing machine learning experiments. For more on Guild AI, visit https://guild.ai.

In this example, we show how Guild is used to run an unmodified Python function as a training operation. It uses a Pythonic interface defined in the module `guild.ipy`.

This notebook covers the following topics:

- Installation
- Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Hyperparameter search and optimization
    - Grid search
    - Random search
    - Bayesian optimization

## Installation

Before continuing, install Guild AI:

    $ pip install guildai
    
Install either `tensorflow` or `tensorflow-gpu` depending on your system capability:

    $ pip install tensorflow # (or tensorflow-gpu if your system has a GPU)
    
TensorFlow is required for TensorBoard support. Your training code does not need to use TensorFlow. This requirement will be removed in versions of Guild starting with `0.7`.
    
**<span style="color:#600">IMPORTANT</span>** If you are installing Guild AI 0.6.3, you must manually install a missing requirement (this bug is fixed in 0.6.4, which will be released in late May):

    $ pip install click
    
## Contents
    
This Notebook has two parts.

It features the module `guild.ipy`, which is a Python interface that can be used interactively in Notebooks or by your own Python code. 

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [1]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [2]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [3]:
import guild.ipy as guild

**<span style="color:#600">IMPORTANT</span>** If you get an error **No module named 'click'** install `click` manually by running `pip install click` in your Notebook environment.

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [4]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir(GUILD_HOME)
    
guild.set_guild_home(GUILD_HOME)

Clear the director of any runs.

In [5]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

Deleted 12 run(s)


## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [6]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

[Function credit: *skopt API documentation*](https://scikit-optimize.github.io/)

Run the mock training function a couple times to see how it works.

**NOTE** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [7]:
print("Trial 1:")
train(x=-2.0)

Trial 1:
loss: 0.018864


In [8]:
print("Trial 2:")
train(x=0.0)

Trial 2:
loss: -0.033283


In [9]:
print("Trial 3:")
train(x=2.0)

Trial 3:
loss: -0.003952


## Generate a run

Run the mock train function using Guild:

In [10]:
run, return_val = guild.run(train)

loss: 0.440338


The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [11]:
print(return_val)

None


The `run` variable is a Python object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [12]:
run.id

'f8fd8c3d6beb4f1d8bd14d6031c2d8c9'

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [13]:
run.dir

'guild-env/runs/f8fd8c3d6beb4f1d8bd14d6031c2d8c9'

Our mock training script doesn't generate any files, so the run directory is empty, with the exception of a `.guild` subdirectory.

In [14]:
os.listdir(run.dir)

['.guild']

The `.guild` subdirectory contains run output and metadata. Outputs include the output printed to the script during the operation (`output`) and any scalars logged (`events.*`). Guild logs scalar output using TensorFlow event files, which can be read using TensorBoard.

In [15]:
os.listdir(os.path.join(run.dir, ".guild"))

['output',
 'events.out.tfevents.1654352263.54f424ef259a.276705.0',
 'opref',
 'attrs']

Run metadata includes *attributes*. You can list the attribute names using `attr_names()`:

In [16]:
run.attr_names()

['exit_status', 'flags', 'id', 'initialized', 'label', 'started', 'stopped']

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [17]:
run.get("flags")

{'noise': 0.1, 'x': 0.1}

Flags are values provided to the function. In this case, both values are defined as function keyword default values. Later you run `train` using explicit flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [18]:
run.status

'completed'

## List runs

List runs using `guild.runs()`:

In [19]:
runs = guild.runs()
runs

,run,operation,started,status,label
0,f8fd8c3d,train(),2022-06-04 14:17:43,completed,noise=0.1 x=0.1


Each time you run an operation, the run appears in the data frame generated by `runs()`.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [20]:
runs.info()

id: f8fd8c3d6beb4f1d8bd14d6031c2d8c9
operation: train()
status: completed
started: 2022-06-04 14:17:43
stopped: 2022-06-04 14:17:43
label: noise=0.1 x=0.1
run_dir: /workspaces/ai_fun/visual-ai-testing/guild-env/runs/f8fd8c3d6beb4f1d8bd14d6031c2d8c9
flags:
  noise: 0.1
  x: 0.1


This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

First, you can specify the `scalars` flag to `info()`:

In [21]:
runs.info(scalars=True)

id: f8fd8c3d6beb4f1d8bd14d6031c2d8c9
operation: train()
status: completed
started: 2022-06-04 14:17:43
stopped: 2022-06-04 14:17:43
label: noise=0.1 x=0.1
run_dir: /workspaces/ai_fun/visual-ai-testing/guild-env/runs/f8fd8c3d6beb4f1d8bd14d6031c2d8c9
flags:
  noise: 0.1
  x: 0.1
scalars:
  loss: 0.440338 (step 0)


Second, you can get a data frame containing scalars using the `scalars()` method.

In [22]:
scalars = runs.scalars()
scalars

,run,prefix,tag,first_val,first_step,last_val,last_step,min_val,min_step,max_val,max_step,avg_val,count,total
0,f8fd8c3d6beb4f1d8bd14d6031c2d8c9,.guild,loss,0.440338,0,0.440338,0,0.440338,0,0.440338,0,0.440338,1,0.440338


**<span style="color:#600">IMPORTANT</span>** If this listing is empty, ensure that you have TensorFlow installed (see above for instructions).

Guild stores aggregates of each tag, including *first*, *last*, *max*, *min*, and *average*.

You can use the various facilities in data frame to get scalar values.

In [23]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

0    0.440338
Name: last_val, dtype: float64

## Generate a second run

Run train again with different flags along with a run *label*:

In [24]:
_ = guild.run(train, x=0.2, _label="run 2")

loss: 0.755420


List runs:

In [25]:
guild.runs()

,run,operation,started,status,label
0,652cc72a,train(),2022-06-04 14:17:45,completed,run 2
1,f8fd8c3d,train(),2022-06-04 14:17:43,completed,noise=0.1 x=0.1


Note that the latest run contains the label "run 2" as specified in the `run()` call.

## Compare runs

Use `compare()` to generate a data frame containing both flags and scalars.

In [26]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,652cc72a,train(),2022-06-04 14:17:45,0 days,completed,run 2,0.1,0.2,0,0.755420
1,f8fd8c3d,train(),2022-06-04 14:17:43,0 days,completed,noise=0.1 x=0.1,0.1,0.1,0,0.440338


## Grid search

Grid search &mdash; also referred to as a parameter sweep &mdash; is a form of hyperparameter tuning that uses exhaustive search over a manually defined set of hyperparameter values.

To perform a grid search in Guild, provide a list of values to use for any given flag. If you specify lists for multiple flags, Guild runs trials for each possible flag value combination.

Run `train()` using a range of values:

In [27]:
_ = guild.run(train, x=[-0.5,-0.4,-0.3,-0.2,-0.1])

Running train (noise=0.1, x=-0.5):
loss: -0.421690
Running train (noise=0.1, x=-0.4):
loss: -0.754186
Running train (noise=0.1, x=-0.3):
loss: -0.816842
Running train (noise=0.1, x=-0.2):
loss: -0.975647
Running train (noise=0.1, x=-0.1):
loss: -0.569882


This command generates five trials, one for each specified value of `x`.

Use `runs().compare()[:5]` to compare the last five results:

In [28]:
guild.runs().compare()[:5]

,run,operation,started,time,status,label,noise,x,step,loss
0,52e33fa2,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.1,0.1,-0.1,0,-0.569882
1,71f410a4,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.2,0.1,-0.2,0,-0.975647
2,6fe5d599,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.3,0.1,-0.3,0,-0.816842
3,942f1309,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.4,0.1,-0.4,0,-0.754186
4,26fd0d1a,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.5,0.1,-0.5,0,-0.421690


View the top-three results by loss:

In [29]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
1,71f410a4,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.2,0.1,-0.2,0,-0.975647
2,6fe5d599,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.3,0.1,-0.3,0,-0.816842
3,942f1309,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.4,0.1,-0.4,0,-0.754186


Based on our mock training function, the "best" result (i.e. the run with the lowest *loss*) should be the run where `x` is close to `-0.3`. Because there's a random component (i.e. the `noise` parameter) your results may show best results with different values for `x`.

Below is an image that plots *loss* for values of *x*, showing the lowest loss where x is approximately `-0.3`.

<img src="bayesian-optimization.png" style="margin-left:0">

[Image credit: *Bayesian optimization with skopt*](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

## Random search

Random search is a method used in machine learning to explore hyperparameter spaces at random.

To run a series of runs using random values over a specified range, use the `slice` function to specify a range for a flag. To specify the number of trials, specify `_max_trials`. The default number of trials is `20`.

Run train five times with random values of `x` over the range `-2.0` to `2.0`.

In [30]:
_ = guild.run(train, x=slice(-2.0,2.0), _max_trials=5)

Running train (noise=0.1, x=1.4244547843294915):
loss: 0.048384
Running train (noise=0.1, x=-0.7948454272117311):
loss: 0.067583
Running train (noise=0.1, x=0.7652979751108959):
loss: -0.372230
Running train (noise=0.1, x=0.5854037787599555):
loss: -0.018798
Running train (noise=0.1, x=1.6300373746484995):
loss: 0.034930


Compare the results again.

In [31]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,0c769e67,train(),2022-06-04 14:17:46,0 days,completed,noise=0.1 x=1.63003,0.1,1.630037,0,0.034930
1,0e6f6330,train(),2022-06-04 14:17:46,0 days,completed,noise=0.1 x=0.58540,0.1,0.585404,0,-0.018798
2,986396f7,train(),2022-06-04 14:17:46,0 days,completed,noise=0.1 x=0.76529,0.1,0.765298,0,-0.372230
3,1c91855f,train(),2022-06-04 14:17:46,0 days,completed,noise=0.1 x=-0.79484,0.1,-0.794845,0,0.067583
4,3e55894a,train(),2022-06-04 14:17:46,0 days,completed,noise=0.1 x=1.42445,0.1,1.424455,0,0.048384
5,52e33fa2,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.1,0.1,-0.100000,0,-0.569882
6,71f410a4,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.2,0.1,-0.200000,0,-0.975647
7,6fe5d599,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.3,0.1,-0.300000,0,-0.816842
8,942f1309,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.4,0.1,-0.400000,0,-0.754186
9,26fd0d1a,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.5,0.1,-0.500000,0,-0.421690


List the three runs with the lowest loss:

In [32]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
6,71f410a4,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.2,0.1,-0.2,0,-0.975647
7,6fe5d599,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.3,0.1,-0.3,0,-0.816842
8,942f1309,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.4,0.1,-0.4,0,-0.754186


## Bayesian optimization

Bayesian optimizers use light weight models as surrogates for the target model — surrogates that can be evaluated quickly to recommend likely optimal hyperparameter values — and update those models using results from real trials.

Run `train` with Guild’s built-in Bayesian optimizer, which uses Gaussian processes. As with the earlier random search, we use Python's slice function to specify the range for `x` over which to search.

**NOTE:** The argument `bayesian` in the command is an alias for `gp`, which is a Bayesian optimizer that uses Gaussian processes. Guild supports three Bayesian optimizers: gp, forest and gbrt.

In [33]:
_ = guild.run(train, x=slice(-2.0,2.0), _optimizer="bayesian", _max_trials=10)

Running train (noise=0.1, x=-1.1758081439824355):
loss: -0.026150
Running train (noise=0.1, x=0.7113735240607029):
loss: -0.114339
Running train (noise=0.1, x=-0.19532758815610585):
loss: -0.661904
Running train (noise=0.1, x=1.9987881431081385):
loss: 0.031694
Running train (noise=0.1, x=-0.13105758798427658):
loss: -0.620627
Running train (noise=0.1, x=-0.38048825224185445):
loss: -0.761236
Running train (noise=0.1, x=-0.4465757653998774):
loss: -0.711492
Running train (noise=0.1, x=-2.0):
loss: 0.093754
Running train (noise=0.1, x=-0.3408864093867128):
loss: -0.762769
Running train (noise=0.1, x=0.21995365949587908):
loss: 0.830106


View the top 10 results:

In [34]:
guild.runs().compare().sort_values(by="loss")[:10]

,run,operation,started,time,status,label,noise,x,step,loss
16,71f410a4,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.2,0.1,-0.200000,0,-0.975647
17,6fe5d599,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.3,0.1,-0.300000,0,-0.816842
1,f71c6966,train(),2022-06-04 14:17:51,0 days,completed,noise=0.1 x=-0.34088,0.1,-0.340886,0,-0.762769
4,1076c27e,train(),2022-06-04 14:17:49,0 days,completed,noise=0.1 x=-0.38048,0.1,-0.380488,0,-0.761236
18,942f1309,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.4,0.1,-0.400000,0,-0.754186
3,b38e184c,train(),2022-06-04 14:17:49,0 days,completed,noise=0.1 x=-0.44657,0.1,-0.446576,0,-0.711492
7,686885f8,train(),2022-06-04 14:17:47,0 days,completed,noise=0.1 x=-0.19532,0.1,-0.195328,0,-0.661904
5,2af9d239,train(),2022-06-04 14:17:48,0 days,completed,noise=0.1 x=-0.13105,0.1,-0.131058,0,-0.620627
15,52e33fa2,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.1,0.1,-0.100000,0,-0.569882
19,26fd0d1a,train(),2022-06-04 14:17:45,0 days,completed,noise=0.1 x=-0.5,0.1,-0.500000,0,-0.421690


## Next steps

This Notebook covers basic functionality provided by the `guild.ipy` module. The module lets you run functions and capture results as unique runs. You can view and compare run results using various module functions.

Generated runs can be further managed using the Guild command line interface. For more information, refer to [Guild AI Quick Start](https://guild.ai/docs/start/).